In [ ]:
pip install twilio

In [ ]:
# 📦 Imports
import pandas as pd
from datetime import datetime
from pathlib import Path
from bs4 import BeautifulSoup
import requests
from google.colab import userdata
from twilio.rest import Client

In [ ]:
# 🔧 Utility Functions

def fetch_price(url, headers, fallback_name):
    title = fallback_name
    price = None
    status = "Unknown"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        title_tag = soup.find('span', id='productTitle')
        price_tag = soup.find('span', class_='a-price-whole')

        title = title_tag.get_text(strip=True) if title_tag else fallback_name
        price_str = price_tag.get_text(strip=True).replace(',', '') if price_tag else 'N/A'

        try:
            price = float(price_str)
            status = "Scraping successful"
        except ValueError:
            price = None
            status = "Scraping successful, price not found/invalid"
    except requests.exceptions.RequestException as e:
        status = f"Scraping failed: {e}"
    except Exception as e:
        status = f"Scraping failed: {e}"

    return title, price, status

def update_price_log(log_file, today, title, price, status):
    if Path(log_file).exists():
        df_log = pd.read_csv(log_file)
    else:
        df_log = pd.DataFrame(columns=['Date', 'Product', 'Price', 'Status', 'URL']) # Added URL column here

    new_entry = pd.DataFrame([[today, title, price, status, url]], columns=['Date', 'Product', 'Price', 'Status', 'URL']) # Added url here
    df_log = pd.concat([df_log, new_entry], ignore_index=True)
    current_entry_index = df_log.index[-1]

    return df_log, current_entry_index

def compare_with_previous(df_log, title, current_entry_index):
    df_product_history = df_log[df_log['Product'] == title].sort_values(by='Date').copy()
    trigger_alert = False
    comparison_status = "Not enough history to compare"
    whatsapp_message = None # Initialize whatsapp_message

    # Safely get the URL for the current entry
    current_url = df_log.loc[current_entry_index, 'URL'] if 'URL' in df_log.columns and current_entry_index in df_log.index else 'N/A'


    if len(df_product_history) > 1:
        last_price_entries = df_product_history['Price'].dropna().iloc[:-1]
        today_price = df_product_history.iloc[-1]['Price']

        if not last_price_entries.empty:
            last_price = last_price_entries.iloc[-1]
            if today_price is not None and last_price is not None: # Added check for None prices
                if today_price < last_price:
                    trigger_alert = True
                    comparison_status = f"Price dropped from ₹{last_price} to ₹{today_price}!"
                    whatsapp_message = f"📉 Amazon Price Drop Alert\n\n{comparison_status}\n\nProduct: {title}\n{current_url}" # Use current_url
                elif today_price > last_price:
                    comparison_status = f"Price increased from ₹{last_price} to ₹{today_price}."
                    # You can add a trigger_alert = True here if you also want notifications for price increases
                    whatsapp_message = f"📈 Amazon Price Increase Alert\n\n{comparison_status}\n\nProduct: {title}\n{current_url}" # Use current_url
                else:
                    comparison_status = "Price remained the same."
                    trigger_alert = True # Trigger alert for same price
                    whatsapp_message = f"↔️ Amazon Price Alert\n\nPrice remained the same at ₹{today_price}.\n\nProduct: {title}\n{current_url}" # Use current_url
            elif today_price is None: # Handle case where today's price is None
                 comparison_status = "Current price not found."
                 whatsapp_message = f"ℹ️ Amazon Price Alert\n\n{comparison_status}\n\nProduct: {title}\n{current_url}" # Use current_url
            elif last_price is None: # Handle case where last price is None
                 comparison_status = "Last price not found, cannot compare."
                 whatsapp_message = f"ℹ️ Amazon Price Alert\n\n{comparison_status}\n\nProduct: {title}\n{current_url}" # Use current_url

        else:
            comparison_status = "Not enough valid price history to compare."
            whatsapp_message = f"ℹ️ Amazon Price Alert\n\n{comparison_status}\n\nProduct: {title}\n{current_url}" # Use current_url
    else:
        # Ensure 5 values are returned even with not enough history
        comparison_status = "Not enough history to compare."
        whatsapp_message = None # No whatsapp message for the first entry


    df_log.loc[current_entry_index, 'Status'] = comparison_status
    return df_log, df_product_history, trigger_alert, comparison_status, whatsapp_message # Return whatsapp_message

def send_whatsapp_alert(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, TWILIO_PHONE_NUMBER, RECIPIENT_PHONE_NUMBER, message):
    try:
        client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

        message = client.messages.create(
            body=message,
            from_=f'whatsapp:{TWILIO_PHONE_NUMBER}',
            to=f'whatsapp:{RECIPIENT_PHONE_NUMBER}'
        )
        return True
    except Exception as e:
        print(f"Error sending WhatsApp message: {e}")
        return False

In [ ]:
from google.colab import files

print("Please upload your product_urls.csv file:")
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# You can now access the uploaded file(s) by their filename(s) in the 'uploaded' dictionary.
# For example, if you uploaded 'product_urls.csv', you can access its content using uploaded['product_urls.csv']

Please upload your product_urls.csv file:


Saving product_urls.csv to product_urls.csv
User uploaded file "product_urls.csv" with length 753 bytes


In [ ]:
# 🚀 Main Execution Loop

# Read Product List
# Read the uploaded product_urls.csv file
try:
    product_list = pd.read_csv('product_urls.csv')  # Attempt to read product_urls.csv
    print("Reading product_urls.csv")
except FileNotFoundError:
    print("product_urls.csv not found. Please upload the file using the cell above.")
    product_list = pd.DataFrame(columns=['Name', 'URL']) # Create an empty DataFrame to avoid errors

# Setup Twilio and Headers
TWILIO_ACCOUNT_SID = userdata.get('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = userdata.get('TWILIO_AUTH_TOKEN')
TWILIO_PHONE_NUMBER = userdata.get('TWILIO_PHONE_NUMBER')
RECIPIENT_PHONE_NUMBER = userdata.get('RECIPIENT_PHONE_NUMBER')

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept-Language': 'en-IN,en;q=0.9'
}

log_file = 'price_log.csv'
today = datetime.now().strftime('%Y-%m-%d')

for _, row in product_list.iterrows():
    url = row['URL']
    name = row['Name']
    print(f"🔍 Checking: {name}")

    title, price, status = fetch_price(url, HEADERS, name)
    df_log, idx = update_price_log(log_file, today, title, price, status)
    df_log, history, alert, msg, whatsapp_message = compare_with_previous(df_log, title, idx) # Added whatsapp_message here
    df_log.to_csv(log_file, index=False)

    print(msg)
    if alert and whatsapp_message: # Check if whatsapp_message is not None
        send_whatsapp_alert(
            TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, TWILIO_PHONE_NUMBER, RECIPIENT_PHONE_NUMBER,
            whatsapp_message
        )

Reading product_urls.csv
🔍 Checking: Samsung Galaxy M14
Price remained the same.
🔍 Checking: Ac
Price remained the same.


In [ ]:
import os
import pandas as pd

# List files in the current directory
print("Files in the current directory:")
print(os.listdir('.'))

# Attempt to read and display product_urls.csv if it exists
file_name = 'product_urls.csv'
if file_name in os.listdir('.'):
    print(f"\nContent of {file_name}:")
    try:
        product_list_check = pd.read_csv(file_name)
        display(product_list_check)
        print(f"\nNumber of rows read: {len(product_list_check)}")
    except Exception as e:
        print(f"Error reading {file_name}: {e}")
else:
    print(f"\n{file_name} not found in the current directory.")

In [51]:
df_log['Date'] = pd.to_datetime(df_log['Date'])
df_log['Price'] = pd.to_numeric(df_log['Price'], errors='coerce')

In [ ]:
import matplotlib.pyplot as plt

unique_products = df_log['Product'].unique()

for product_name in unique_products:
    df_product = df_log[df_log['Product'] == product_name].copy()
    df_product.dropna(subset=['Price'], inplace=True) # Drop rows where Price is NaN for plotting

    if not df_product.empty:
        fig, ax = plt.subplots(figsize=(10, 6)) # Adjust figure size as needed
        df_product.plot(x='Date', y='Price', ax=ax, marker='o') # Added marker for better visibility

        ax.set_title(f'Price Trend for {product_name}')
        ax.set_xlabel('Date')
        ax.set_ylabel('Price')
        ax.grid(True)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print(f"No valid price data to plot for {product_name}")
